# Spectrum Licence Composite Dataset Construction

The Register of Radiocommunication Licences (RRL) is database containing licensing information spread across various .csv files. The information stored in the various csv files need to be merged into a single dataset using a licence number index.

### Desire Spectrum Licence Information
- Licence number
- Client number
- Frequency Ranges
- Bandwidth
- HCIS Cells

In [7]:
import pandas as pd
import numpy as np

# getRRL() -> To be implemented. Downloads RRL and verifies structure is compatible with specNav.

# -----------------
# def getSpecLic ():
# -----------------

licence = pd.read_csv('licence.csv')


# Column tidy
licence.drop(columns=[
    'SV_ID',
    'SS_ID',
    'DATE_ISSUED',
    'DATE_OF_EFFECT',
    'DATE_OF_EXPIRY',
    'STATUS',
    'STATUS_TEXT',
    'AP_ID',
    'AP_PRJ_IDENT',
    'SHIP_NAME',
    'BSL_NO'
    ], inplace=True)

# Isolate Spectrum Licence
specLicence = licence.loc[licence['LICENCE_TYPE_NAME'] == 'Spectrum']
specLicence.reset_index(drop=True, inplace=True)
specLicence
    

,LICENCE_NO,CLIENT_NO,LICENCE_TYPE_NAME,LICENCE_CATEGORY_NAME
0,10143110,1136980.0,Spectrum,2 GHz Band
1,10143136,1103274.0,Spectrum,2 GHz Band
2,10143466,1104504.0,Spectrum,2 GHz Band
3,10143562,1103276.0,Spectrum,2 GHz Band
4,10231258,1103276.0,Spectrum,1800 MHz Band
...,...,...,...,...
161,9469870,1149289.0,Spectrum,2.5 GHz Band
162,9469871,1103275.0,Spectrum,2.5 GHz Band
163,9469878,1103275.0,Spectrum,2.5 GHz Band
164,9599659,1103275.0,Spectrum,2.3 GHz Band
